In [39]:
class spacy_tokenizer():
    def __init__(self):
        self
    def __call__(self,post):
        doc = nlp(post)
        return " ".join(token.lemma_ for token in doc if 
                         token.pos_.lower() not in ['aux','punct','cconj','det','space','conj','adp','pron','sym'])

In [40]:
tvec = TfidfVectorizer(stop_words = stopwords, 
                       max_features = 1000, 
                       min_df = 2,
                       max_df = .9,
                       ngram_range = (1,1),
                       tokenizer = spacy_tokenizer())

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer

In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
from sklearn.model_selection import GridSearchCV, train_test_split

In [5]:
from sklearn.pipeline import Pipeline

In [6]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

In [7]:
from sklearn.neighbors import KNeighborsClassifier

In [8]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [9]:
import xgboost as xg

In [10]:
import spacy

In [11]:
from nltk.corpus import stopwords

In [12]:
from nltk import word_tokenize 

In [13]:
from nltk.stem import WordNetLemmatizer

In [14]:
posts = pd.read_csv('posts.csv')

In [15]:
X = posts['selftext']
y = posts['target']

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    stratify = y, 
                                                    random_state = 42)

In [17]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((18522,), (18522,), (6175,), (6175,))

In [18]:
y_train.value_counts(normalize=True)

0    0.693284
1    0.306716
Name: target, dtype: float64

In [ ]:
X_train = tvec.fit_transform(X_train)
X_test = tvec.transform(X_test)

In [112]:
X_train = pd.DataFrame(X_train.toarray(), columns = tvec.get_feature_names_out())
X_test = pd.DataFrame(X_test.toarray(), columns = tvec.get_feature_names_out())

In [22]:
model_dict = {'mnb': # 1st level 1st key
              {'model': # 2nd level 1st key
               ('mnb', MultinomialNB()),
               'params':{}
              },
  #            'knn': # 1st level 2nd key
   #           {'model': # 2nd level 1st key
    #           ('knn', KNeighborsClassifier()),
     #          'params':# 2nd level 2nd key
      #         {"knn__n_neighbors": [3,5,7,10],
       #         "knn__p":[1,2,3]}},
             'logr': # 1st level 2nd key
              {'model': # 2nd level 1st key
               ('logr', LogisticRegression()),
               'params':# 2nd level 2nd key
               {"logr__C": [1,10,100]}},
#             'abc': # 1st level 2nd key
 #             {'model': # 2nd level 1st key
  #             ('abc', AdaBoostClassifier()),
   #            'params':# 2nd level 2nd key
    #           {"abc__n_estimators": [30,50,100],
     #           "abc__learning_rate": [0.2,0.4,0.6]}}
             }

In [42]:
# set dictionary to house best estimators and params
best_models = {}
# for loop through the 3 1st level keys of model_model dict
for key, value in model_dict.items():
    # set pipeline with Count Vectorizer
    pipe = Pipeline([('tvec', TfidfVectorizer(stop_words = stopwords, 
                                              max_features = 1000, 
                                              min_df = 2,
                                              max_df = .9,
                                              ngram_range = (1,1),
                                              tokenizer = LemmaTokenizer()))])
    # append current model iteration tuple
    pipe.steps.append(value['model'])
    print(pipe)
    gs = GridSearchCV(pipe, # set as iteration of pipe
                  value['params'], # set as iteration of pipe_params
                      cv=5, # Cross Validation of 5
                      n_jobs=-1) # Unlock CPU to help process
    # fit the current iteration of GridSearchCV
    gs.fit(X_train, y_train)
    # save the best model, best params, and the pipe for scoring 
    best_models[key] = {'model': gs.best_estimator_,
                        'params': gs.best_params_,
                        'pipe': pipe}
    print(best_models)

NameError: name 'model_dict' is not defined

In [43]:
for key, value in best_models.items():
    value['model'].fit(X_train,y_train)
    print(value['model'].score(X_train, y_train), value['model'].score(X_test, y_test))

# LogReg

In [26]:
#logr_params = {
 #   'logr__C':[0.1,0.5,0.9,1]
#}

In [38]:
gs = GridSearchCV(LogisticRegression(),
                  param_grid = {'C':[0.01,0.1,1,10,100]},
                  cv=5,
                 verbose=1,
                 n_jobs = -1)
gs.fit(X_train, y_train)
logr = gs.best_estimator_
logr.fit(X_train,y_train)
logr.score(X_train, y_train), logr.score(X_test, y_test)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


(0.8840837922470576, 0.8741700404858299)

#  MNB

In [39]:
gs = GridSearchCV(MultinomialNB(),
                  param_grid = {'alpha':[0.1,0.5,0.9,1]},
                  cv=5,
                 verbose=1,
                 n_jobs = -1)
gs.fit(X_train, y_train)
mnb = gs.best_estimator_
mnb.fit(X_train,y_train)
mnb.score(X_train, y_train), mnb.score(X_test, y_test)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


(0.8284742468415938, 0.8225101214574899)

# Gaussian NB

In [40]:
gs = GridSearchCV(GaussianNB(),
                  param_grid = {'var_smoothing':[1e-7,1e-8,1e-9,1e-10]},
                  cv=5,
                 verbose=1,
                 n_jobs = -1)
gs.fit(X_train, y_train)
gnb = gs.best_estimator_
gnb.fit(X_train,y_train)
gnb.score(X_train, y_train), gnb.score(X_test, y_test)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


(0.7540222438181622, 0.7462348178137652)

# Bernoulli NB

# KNN

In [41]:
gs = GridSearchCV(KNeighborsClassifier(),
                  param_grid = {'n_neighbors':[3,5,7,10,15]},
                  cv=5,
                 verbose=1,
                 n_jobs = -1)
gs.fit(X_train, y_train)
knn = gs.best_estimator_
knn.fit(X_train,y_train)
knn.score(X_train, y_train), knn.score(X_test, y_test)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Anaconda\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avo

(0.7978080120937264, 0.7791093117408907)

# Decision Tree

In [42]:
gs = GridSearchCV(DecisionTreeClassifier(),
                  param_grid = {'max_depth':[3,5,7]},
                  cv=5,
                 verbose=1,
                 n_jobs = -1)
gs.fit(X_train, y_train)
dt = gs.best_estimator_
dt.fit(X_train,y_train)
dt.score(X_train, y_train), dt.score(X_test, y_test)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


(0.8306878306878307, 0.828663967611336)

# Random Forest

In [43]:
gs = GridSearchCV(RandomForestClassifier(),
                  param_grid = {
                      'n_estimators':[30,50,100],
                      'max_depth':[3,5,7]},
                  cv=5,
                 verbose=1,
                 n_jobs = -1)
gs.fit(X_train, y_train)
rf = gs.best_estimator_
rf.fit(X_train,y_train)
rf.score(X_train, y_train), rf.score(X_test, y_test)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


(0.7799913616240147, 0.7721457489878543)

# AdaBoost

In [53]:
gs = GridSearchCV(AdaBoostClassifier(),
                  param_grid = {
                      'n_estimators':[30,50,100,200]},
                  cv=5,
                 verbose=1,
                 n_jobs = -1)
gs.fit(X_train, y_train)
abc = gs.best_estimator_.fit
abc.fit(X_train,y_train)
abc.score(X_train, y_train), abc.score(X_test, y_test)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


(0.8687506748731239, 0.8607287449392712)

# XGBoost

In [65]:
import re

In [77]:
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

In [78]:
# cite source
[regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X_train.columns]

['!',
 '#',
 '$',
 '%',
 '&',
 "'",
 "''",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '--',
 '.',
 '..',
 '...',
 '....',
 '/',
 '//www.youtube.com/watch',
 '1',
 '1.',
 '1/2',
 '10',
 '100',
 '15',
 '2',
 '2.',
 '20',
 '3',
 '3.',
 '30',
 '4',
 '5',
 '50',
 '6',
 '7',
 '8',
 ':',
 ';',
 '?',
 '@',
 '_',
 '\\',
 '\\-',
 '_',
 '``',
 'able',
 'absolutely',
 'abuse',
 'according',
 'acid',
 'action',
 'active',
 'activism',
 'activist',
 'actual',
 'actually',
 'add',
 'added',
 'adding',
 'advance',
 'advice',
 'age',
 'ago',
 'agree',
 'agriculture',
 'allergic',
 'allergy',
 'allowed',
 'almond',
 'alternative',
 'amazing',
 'american',
 'amp',
 'animal',
 'answer',
 'anxiety',
 'anybody',
 'anymore',
 'anyways',
 'app',
 'apparently',
 'apple',
 'appreciate',
 'appreciated',
 'approach',
 'area',
 'arenâ\x80\x99t',
 'argument',
 'article',
 'ask',
 'asked',
 'asking',
 'assume',
 'ate',
 'auto=webp',
 'available',
 'average',
 'avocado',
 'avoid',
 'aware',
 'away',
 'b12',
 'baby',
 'baco

In [44]:
gs = GridSearchCV(xg.XGBClassifier(),
                  param_grid = {
                      'n_estimators':[30,50,100,200]},
                  cv=5,
                 verbose=1,
                 n_jobs = -1)
gs.fit(X_train, y_train)
xgb = gs.best_estimator_.fit
xgb.fit(X_train,y_train)
xgb.score(X_train, y_train), xgb.score(X_test, y_test)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


KeyboardInterrupt: 

# Stacking

In [44]:
from sklearn.ensemble import StackingClassifier

In [51]:
stack = StackingClassifier(estimators = [('logr',logr),
                                         ('mnb',mnb),
                                         ('knn',knn),
                                         ('dt',dt)
                                        ])

In [52]:
stack.fit(X_train,y_train)
stack.score(X_train,y_train),stack.score(X_test,y_test)

(0.8907245437857683, 0.88)